In [1]:
setwd("/data/Projects/ACC_NCDR/NCDR/BJMDATA/ACTION/NCHData/multiple_imputed/")

In [2]:
library(caret)

load(file ='/data/Projects/ACC_NCDR/NCDR/BJMDATA/ACTION/NCHData/multiple_imputed/all_imputed_folds.RData')

Loading required package: lattice
Loading required package: ggplot2


In [4]:
folds.all <- list(fold.1, fold.2, fold.3, fold.4, fold.5)

In [18]:
fold <- 5
set.seed(fold*fold)
model.data <- folds.all[[fold]]


In [19]:
#remove outcome (here, DCStatus) and other unneeded columns from training data
data.labels <- labels
hospital.events <- names(model.data)[which(grepl("CE", names(model.data)))]
hospital.events <- hospital.events[-c(6,12)]
model.data <- model.data[, which(!(colnames(model.data) %in% c("SubmissionPatientKey", "DCStatus", hospital.events)))]

#scale
#Note- Comment out the following line to produce raw data.
#      Run it to produce scaled (normalized?) data.
model.data <- as.data.frame(scale(model.data))
ID.vector <- c(1:dim(model.data)[1])

In [20]:
#partition data into 40:40:20 via 2 splits, this one for 40:60
Split1 <- createDataPartition(y = data.labels, p = 0.4, list = FALSE)
trainA <- model.data[Split1,]
labelsA <- data.labels[Split1]
trainA$DCStatus <- labelsA
trainA$IDX <- ID.vector[Split1]

#this one for 60 into 40:20
remainder.df <- model.data[-Split1,]
remainder.labels <- data.labels[-Split1]
remainder.ID <- ID.vector[-Split1]

Split2 <- createDataPartition(y = remainder.labels, p = (2/3), list = FALSE)
trainB <- remainder.df[Split2,]
labelsB <- remainder.labels[Split2]
trainB$DCStatus <- labelsB
trainB$IDX <- remainder.ID[Split2]

trainC <- remainder.df[-Split2,]
labelsC <- remainder.labels[-Split2]
trainC$DCStatus <- labelsC
trainC$IDX <- remainder.ID[-Split2]

In [21]:
write.csv(trainA, file = sprintf('fold_%s/trainA_imp.csv', fold), row.names = FALSE)
write.csv(trainB, file = sprintf('fold_%s/trainB_imp.csv', fold), row.names = FALSE)
write.csv(trainC, file = sprintf('fold_%s/trainC_imp.csv', fold), row.names = FALSE)

In [ ]:
#model.data$IDX <- ID.vector

#model.data$DCStatus <- data.labels

#write.csv(model.data, file = 'all_no_imp.csv', row.names = FALSE)